<a href="https://colab.research.google.com/github/tosaka-naoya/carefree/blob/main/%E7%B7%9A%E5%BD%A2%E8%A8%88%E7%94%BB%E6%B3%95%E3%81%AE%E7%B7%B4%E7%BF%923.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ライブラリのインストール
!pip install pulp
import pulp
from pulp import *

#問題の定義
m = LpProblem(name="XとYの価格の合計最大化", sense=LpMaximize)

#変数の定義
x = LpVariable("x", lowBound=0, cat="Integer")
y = LpVariable("y", lowBound=0, cat="Integer")

#目的関数の定義
m += 100*x + 100*y

#制約条件の定義
m += x + 2*y <= 16
m += 3*x + y <= 18

#ソルバーの実装
m.solve()

#解の表示
print()
print("x:", value(x), " ", "y:", value(y))


x: 4.0   y: 6.0


In [2]:
#ライブラリのインポート
import numpy as np
import pandas as pd
from itertools import product

#パラメータの設定
np.random.seed(1)
nw, nf = 3, 4
pr = list(product(range(nw),range(nf)))
Supply = np.random.randint(30, 50, nw)
Demand = np.random.randint(20, 30, nf)
Transport_Cost = np.random.randint(10, 20, (nw,nf))

In [3]:
#pandasを使わない場合

#問題の定義
m1 = LpProblem()

#変数の定義
v1 = {(i,j):LpVariable('v%d_%d'%(i,j), lowBound=0) for i,j in pr}

#目的関数の定義
m1 += lpSum(Transport_Cost[i][j] * v1[i,j] for i,j in pr)

#制約条件の定義
for i in range(nw):
  m1 += lpSum(v1[i,j] for j in range(nf)) <= Supply[i]
for j in range(nf):
  m1 += lpSum(v1[i,j] for i in range(nw)) >= Demand[j]

#ソルバーの実装
m1.solve()

#結果表示
{k:value(x) for k,x in v1.items() if value(x) > 0}

{(0, 0): 28.0,
 (0, 1): 7.0,
 (1, 1): 22.0,
 (1, 3): 3.0,
 (2, 2): 25.0,
 (2, 3): 17.0}

In [4]:
#pandasを使う場合

#表の作成
a = pd.DataFrame([(i,j) for i,j in pr], columns=['Warehouse','Factory'])
a['Transport_Cost'] = Transport_Cost.flatten()
display(a[:3])

,Warehouse,Factory,Transport_Cost
0,0,0,10
1,0,1,11
2,0,2,17


In [5]:
#問題の定義
m2 = LpProblem()

#変数の定義
a['Var'] = [LpVariable('v%d'%i, lowBound=0) for i in a.index]

#目的関数の定義
m2 += lpDot(a.Transport_Cost, a.Var)

#制約条件の定義
for k, v in a.groupby('Warehouse'):
    m2 += lpSum(v.Var) <= Supply[k]
for k, v in a.groupby('Factory'):
    m2 += lpSum(v.Var) >= Demand[k]

#ソルバーの実装
m2.solve()

#結果表示
a['Val'] = a.Var.apply(value)
a[a.Val > 0]

,Warehouse,Factory,Transport_Cost,Var,Val
0,0,0,10,v0,28.0
1,0,1,11,v1,7.0
5,1,1,12,v5,22.0
7,1,3,15,v7,3.0
10,2,2,12,v10,25.0
11,2,3,14,v11,17.0


このサイトの練習問題を少しアレンジして写経
https://qiita.com/SaitoTsutomu/items/070ca9cb37c6b2b492f0